In [1]:
from pamda import pamda

In [2]:
from math import log

def bmssp_big_o(n, m):
    # BMSSP has a time complexity of O(m log(n)^(2/3))
    return n+m

def dijkstra_big_o(n, m):
    # Dijkstra's algorithm as implemented in scgraph has a time complexity of O((n + m) log(n)
    return (m+n)*log(n)

In [3]:
benchmark_data =pamda.read_csv('./outputs/gridscale_time_tests.csv')

for row in benchmark_data:
    n_nodes = int(row['graph_nodes'])
    n_edges = int(row['graph_edges'])
    row['exp_bmssp_to_dijkstra_performance_ratio'] =  bmssp_big_o(n_nodes, n_edges) / dijkstra_big_o(n_nodes, n_edges)
    row['real_bmssp_to_dijkstra_performance_ratio'] = float(row['bmssp_spantree_time_ms']) / float(row['sc_dijkstra_spantree_time_ms'])
    row['pred_bmssp_breakeven_multiplier'] = row['exp_bmssp_to_dijkstra_performance_ratio'] / row['real_bmssp_to_dijkstra_performance_ratio']
    print(row['graph_nodes'], row['pred_bmssp_breakeven_multiplier'])

1089 0.006413981914440669
1089 0.00706662794135963
1089 0.006208066964909857
2500 0.005960512524750273
2500 0.0028254560359469404
2500 0.004449354739016642
10000 0.008153778187107061
10000 0.005533901652331875
10000 0.009031994534584151
40000 0.0070930306991738665
40000 0.007233029811888584
40000 0.008050194704959112
160000 0.00722055142741702
160000 0.008683940296752552
160000 0.008186878047965017
640000 0.008056158079188075
640000 0.008723085947095678
640000 0.008431053937284735
2560000 0.007506795100412549
2560000 0.008090829344864266
2560000 0.008225667259457416
10240000 0.007694341098130258
10240000 0.008528285072118407
10240000 0.008231932278122913


In [4]:

bmssp_dijkstra_big_o_ratios = []
for magnitude in list(range(1,10)) + list(range(10,300,10)):
    n = 10**magnitude
    connections_per_node = 8
    m = connections_per_node*n
    out ={
        'n_magnitude': f"1e{magnitude}",
        'connections_per_node': connections_per_node,
        'bmssp_to_dijkstra_ratio': bmssp_big_o(n,m)/dijkstra_big_o(n,m)
    }
    bmssp_dijkstra_big_o_ratios.append(out)
    print(out)

pamda.write_csv(filename='./outputs/bmssp_dijkstra_big_o_ratios.csv', data=bmssp_dijkstra_big_o_ratios)


{'n_magnitude': '1e1', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.43429448190325176}
{'n_magnitude': '1e2', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.2171472409516259}
{'n_magnitude': '1e3', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.14476482730108395}
{'n_magnitude': '1e4', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.10857362047581294}
{'n_magnitude': '1e5', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.08685889638065036}
{'n_magnitude': '1e6', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.07238241365054197}
{'n_magnitude': '1e7', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.06204206884332169}
{'n_magnitude': '1e8', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.05428681023790647}
{'n_magnitude': '1e9', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.048254942433694645}
{'n_magnitude': '1e10', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.04342944819032518}
{'n_magnitude': '1e

In [5]:
from math import exp

pred_breakeven_multiplier = sum([row['pred_bmssp_breakeven_multiplier'] for row in benchmark_data]) / len(benchmark_data)
print(f"Predicted breakeven at multiplier = {pred_breakeven_multiplier}")

# TODO: Validate math here.

# Math derivation:
#
# BMSSP(n, m) = m * (ln n)^(2/3)
# Dijkstra(n, m) = (n + m) * ln n
#
# Assume m = 8n:
#   8n * (ln n)^(2/3) = p * (9n * ln n)
#
# Cancel n:
#   8 * (ln n)^(2/3) = 9p * ln n
#
# Divide by (ln n)^(2/3):
#   8 = 9p * (ln n)^(1/3)
#
# Solve for ln n:
#   (ln n)^(1/3) = 8 / (9p)
#   ln n = (8 / (9p))^3
#
# Finally, to get log10(n):
#   log10(n) = ln(n) / ln(10)

breakeven_magnitude_n = ((8 / (9 * pred_breakeven_multiplier)) ** (3)) / log(10)

print(f"Predicted breakeven at n = 1e{breakeven_magnitude_n} nodes")


Predicted breakeven at multiplier = 0.007316643649969898
Predicted breakeven at n = 1e778737.4002938061 nodes
